<br><br>

# 下面这行代码里的邮箱和密码，要改为自己的邮箱和密码

<br><br>

In [ ]:
import requests
import json
from os.path import expanduser
from requests.auth import HTTPBasicAuth

# 创建会话对象
sess = requests.Session()

# 设置基本身份验证
sess.auth = HTTPBasicAuth('邮箱', '密码')

# 向API发送POST请求进行身份验证
response = sess.post('https://api.worldquantbrain.com/authentication')

# 打印响应状态和内容以调试
print(response.status_code)
print(response.json())

In [2]:
simulation_data = {
    'type': 'REGULAR',
    'settings': {
        'instrumentType': 'EQUITY',
        'region': 'USA',
        'universe': 'TOP3000',
        'delay': 1,
        'decay': 0,
        'neutralization': 'SUBINDUSTRY',
        'truncation': 0.08,
        'pasteurization': 'ON',
        'unitHandling': 'VERIFY',
        'nanHandling': 'OFF',
        'language': 'FASTEXPR',
        'visualization': False,
    },
    'regular': 'liabilities/assets'  ## 写表达式
}

from time import sleep

sim_resp = sess.post(
    'https://api.worldquantbrain.com/simulations',
    json=simulation_data,
)

sim_progress_url = sim_resp.headers['Location']

while True:
    sim_progress_resp = sess.get(sim_progress_url)
    retry_after_sec = float(sim_progress_resp.headers.get("Retry-After", 0))
    if retry_after_sec == 0:  # simulation done!模拟完成!
        break
    sleep(retry_after_sec)

alpha_id = sim_progress_resp.json()["alpha"]  # the final simulation result 模拟最终模拟结果

print(alpha_id)

2lVOkw6


In [3]:
# 获取数据集ID为fundamental6（Company Fundamental Data for Equity）下的所有数据字段
### Get Data_fields like Data Explorer 获取所有满足条件的数据字段及其ID
def get_datafields(
        s,
        searchScope,
        dataset_id: str = '',
        search: str = ''
):
    import pandas as pd
    instrument_type = searchScope['instrumentType']
    region = searchScope['region']
    delay = searchScope['delay']
    universe = searchScope['universe']

    if len(search) == 0:
        url_template = "https://api.worldquantbrain.com/data-fields?" + \
                       f"&instrumentType={instrument_type}" + \
                       f"&region={region}&delay={str(delay)}&universe={universe}&dataset.id={dataset_id}&limit=50" + \
                       "&offset={x}"
        count = s.get(url_template.format(x=0)).json()['count']
    else:
        url_template = "https://api.worldquantbrain.com/data-fields?" + \
                       f"&instrumentType={instrument_type}" + \
                       f"&region={region}&delay={str(delay)}&universe={universe}&limit=50" + \
                       f"&search={search}" + \
                       "&offset={x}"
        count = 100

    datafields_list = []
    for x in range(0, count, 50):
        datafields = s.get(url_template.format(x=x))
        datafields_list.append(datafields.json()['results'])

    datafields_list_flat = [item for sublist in datafields_list for item in sublist]

    datafields_df = pd.DataFrame(datafields_list_flat)
    return datafields_df

# 爬取id
searchScope = {'region': 'USA', 'delay': '1', 'universe': 'TOP3000', 'instrumentType': 'EQUITY'}
fundamental6 = get_datafields(s=sess, searchScope=searchScope, dataset_id='analyst4') # id设置

# 筛选（这里是type的MATRIX）
fundamental6 = fundamental6[fundamental6['type'] == "MATRIX"]
fundamental6.head()

datafields_list_fundamental6 = fundamental6['id'].values
print(datafields_list_fundamental6[:5])
print(len(datafields_list_fundamental6))

['anl4_adjusted_netincome_ft' 'anl4_bvps_flag' 'anl4_bvps_high'
 'anl4_bvps_low' 'anl4_bvps_mean']
202


In [4]:
# 将datafield替换到Alpha模板(框架)中group_rank({fundamental model data}/cap,subindustry)批量生成Alpha
alpha_list = []

for index,datafield in enumerate(datafields_list_fundamental6,start=1):
    print(f"正在循环第 {index} 个元素")
    print("正在将如下alpha表达式与setting封装")
    #alpha_expression = f'group_rank(({datafield})/cap, subindustry)'
    alpha_expression = f'trade_when(close>ts_mean(close,90),{datafield},-1)'
    print(alpha_expression)
    simulation_data = {
        "type": "REGULAR",
        "settings": {
            "instrumentType": "EQUITY",
            "region": "USA",
            "universe": "TOP3000",
            "delay": 1,
            "decay": 0,
            "neutralization": "SUBINDUSTRY",
            "truncation": 0.02,
            "pasteurization": "ON",
            "unitHandling": "VERIFY",
            "nanHandling": "ON",
            "language": "FASTEXPR",
            "visualization": False,
        },
        "regular": alpha_expression
    }
    alpha_list.append(simulation_data)
    print(f"there are {len(alpha_list)} Alphas to simulate")

    
for index,datafield in enumerate(datafields_list_fundamental6,start=1):
    print(f"正在循环第 {index} 个元素")
    print("正在将如下alpha表达式与setting封装")
    #alpha_expression = f'group_rank(({datafield})/cap, subindustry)'
    alpha_expression = f'trade_when(close>ts_mean(close,90),ts_rank({datafield},90),-1)'
    #trade_when(close>ts_mean(close,300),-scl12_buzz,-1) 
    print(alpha_expression)
    simulation_data = {
        "type": "REGULAR",
        "settings": {
            "instrumentType": "EQUITY",
            "region": "USA",
            "universe": "TOP3000",
            "delay": 1,
            "decay": 0,
            "neutralization": "SUBINDUSTRY",
            "truncation": 0.02,
            "pasteurization": "ON",
            "unitHandling": "VERIFY",
            "nanHandling": "ON",
            "language": "FASTEXPR",
            "visualization": False,
        },
        "regular": alpha_expression
    }
    alpha_list.append(simulation_data)
    print(f"there are {len(alpha_list)} Alphas to simulate")

print(alpha_list[1])

正在循环第 1 个元素
正在将如下alpha表达式与setting封装
trade_when(close>ts_mean(close,90),anl4_adjusted_netincome_ft,-1)
there are 1 Alphas to simulate
正在循环第 2 个元素
正在将如下alpha表达式与setting封装
trade_when(close>ts_mean(close,90),anl4_bvps_flag,-1)
there are 2 Alphas to simulate
正在循环第 3 个元素
正在将如下alpha表达式与setting封装
trade_when(close>ts_mean(close,90),anl4_bvps_high,-1)
there are 3 Alphas to simulate
正在循环第 4 个元素
正在将如下alpha表达式与setting封装
trade_when(close>ts_mean(close,90),anl4_bvps_low,-1)
there are 4 Alphas to simulate
正在循环第 5 个元素
正在将如下alpha表达式与setting封装
trade_when(close>ts_mean(close,90),anl4_bvps_mean,-1)
there are 5 Alphas to simulate
正在循环第 6 个元素
正在将如下alpha表达式与setting封装
trade_when(close>ts_mean(close,90),anl4_bvps_median,-1)
there are 6 Alphas to simulate
正在循环第 7 个元素
正在将如下alpha表达式与setting封装
trade_when(close>ts_mean(close,90),anl4_bvps_number,-1)
there are 7 Alphas to simulate
正在循环第 8 个元素
正在将如下alpha表达式与setting封装
trade_when(close>ts_mean(close,90),anl4_bvps_value,-1)
there are 8 Alphas to simulate
正在循环第 9 个元素
正在将如

In [5]:
print(alpha_list[1])

{'type': 'REGULAR', 'settings': {'instrumentType': 'EQUITY', 'region': 'USA', 'universe': 'TOP3000', 'delay': 1, 'decay': 0, 'neutralization': 'SUBINDUSTRY', 'truncation': 0.02, 'pasteurization': 'ON', 'unitHandling': 'VERIFY', 'nanHandling': 'ON', 'language': 'FASTEXPR', 'visualization': False}, 'regular': 'trade_when(close>ts_mean(close,90),anl4_bvps_flag,-1)'}


In [6]:
##################测试前两个##############################
from time import sleep

for index,alpha in enumerate(alpha_list[0:2],start=1):
    sim_resp = sess.post(
        'https://api.worldquantbrain.com/simulations',
        json=alpha,
    )

    try:
        sim_progress_url = sim_resp.headers['Location']
        while True:
            sim_progress_resp = sess.get(sim_progress_url)
            retry_after_sec = float(sim_progress_resp.headers.get("Retry-After", 0))
            if retry_after_sec == 0:  # simulation done!模拟完成!
                break
            sleep(retry_after_sec)
        alpha_id = sim_progress_resp.json()["alpha"]  # the final simulation result.# 最终模拟结果
        print(alpha_id)
    except:
        print("no location, sleep for 10 seconds and try next alpha.“没有位置，睡10秒然后尝试下一个字母。”")
        sleep(10)

#################################################

r53WvQo
39bALZz


In [ ]:
# 将Alpha一个一个发送至服务器进行回测（已经测试前两个了）
from time import sleep

for index,alpha in enumerate(alpha_list,start=1):
    sim_resp = sess.post(
        'https://api.worldquantbrain.com/simulations',
        json=alpha,
    )

    try:
        sim_progress_url = sim_resp.headers['Location']
        while True:
            sim_progress_resp = sess.get(sim_progress_url)
            retry_after_sec = float(sim_progress_resp.headers.get("Retry-After", 0))
            if retry_after_sec == 0:  # simulation done!模拟完成!
                break
            sleep(retry_after_sec)
        alpha_id = sim_progress_resp.json()["alpha"]  # the final simulation result.# 最终模拟结果
        print(alpha_id)
    except:
        print("no location, sleep for 10 seconds and try next alpha.“没有位置，睡10秒然后尝试下一个字母。”")
        sleep(10)

r53WvQo
39bALZz
bRk9pRr
dOXQeEX
zYMWwao
xY6n9vn
0Rl8Qb2
mg3XzNK
RV3rd2d
39bAAzQ
e9prOmE
xY6nxJq
5QbvOX1
9VvR73o
pPX7jAj
M1Wx1x9
mg3X6Rx
gQA3ZLQ
9VvRExd
39bAoK0
Ev7KmZJ
GdaoQRx
Ev7K8Xr
39bAPEZ
dOXQo0K
nKJWrLw
Gdao72o
Q3JQpGM
o6pYj9l
Gdao28J
dOXQGLX
9VvRnkr
9VvRNRd
RV3pJW1
OrYp0xd
NQwpxqo
39b7w1P
dOXxVM2
Ev7wJ5m
nKJgb2x
wYmR6El
OrYpKog
A0WwJ8Y
aLYdlLv
9Vvw6OK
KO0bXJl
7ZAwd3L
